# Chatbot with Memory

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

In [6]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is often associated with tranquility, stability, and peace. It is a calming color that can promote relaxation and a sense of serenity. Blue is also commonly chosen as a favorite color by many people, as it is versatile and can be found in various shades and tones. Whether it's a deep navy blue, a bright sky blue, or a soft baby blue, there is a shade of blue for everyone to enjoy.", response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 13, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e82db82a-e5e4-4da3-99e5-f4d0da139cef-0', usage_metadata={'input_tokens': 13, 'output_tokens': 87, 'total_tokens': 100})

In [7]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, but I am unable to determine your favorite color as I do not have access to personal information about you. Can you please tell me your favorite color?", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 13, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cbbab7f8-5402-499c-b6b9-c97c6975ce19-0', usage_metadata={'input_tokens': 13, 'output_tokens': 35, 'total_tokens': 48})

## Let's add memory to our Chatbot

In [8]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [9]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [10]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [11]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [12]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [13]:
chain.invoke("my name is Bappy")

{'content': 'my name is Bappy',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Nice to meet you, Bappy! How can I help you today?'}

In [14]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is Bappy'),
  AIMessage(content='Nice to meet you, Bappy! How can I help you today?')],
 'text': 'Your name is Bappy.'}